In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import joblib
import warnings
warnings.filterwarnings("ignore")


def load_model(path=''):
  return joblib.load(path)

In [ ]:
lr_model=load_model(r"/content/drive/MyDrive/AutoEncoders/lr_file_encoder.pkl")

In [ ]:
!pip install bnlp-toolkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 43.2 MB/s 
     |████████████████████████████████| 23.9 MB 1.2 MB/s 
     |████████████████████████████████| 965 kB 58.3 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import pandas as pd
import numpy as np
import re
from bnlp.corpus import stopwords
from sklearn.feature_extraction.text import HashingVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import *

%matplotlib inline
sns.set()

punkt not found. downloading...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
label_auth=pd.read_csv(r'/content/drive/MyDrive/AutoEncoders/LabeledAuthentic-7K.csv') #, nrows=3067)
label_fake=pd.read_csv(r'/content/drive/MyDrive/AutoEncoders/LabeledFake-1K.csv') #, nrows=995)
label_auth.drop(['source','relation'], axis=1, inplace=True)
label_fake.drop(['source','relation','F-type'], axis=1, inplace=True)
df=pd.concat([label_auth, label_fake], axis=0)
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
df['label']=np.where(df['label']==0.0, 0, 1)
df.tail()
df=df.sample(frac=1).reset_index(drop=True)
print(df.shape)
df.head()

(8501, 7)


,articleID,domain,date,category,headline,content,label
0,5214,channelionline.com,2018-09-19 21:02:35,Entertainment,বিতর্কের মুখে পরাজয় মানলেন সালমান!,নিজের প্রযোজিত সিনেমা ‘লাভরাত্রি’র নাম নিয়ে মা...,1
1,6205,dhakatimes24.com,2018-09-20 07:45:58,Miscellaneous,"নড়িয়া নিয়ে প্রধানমন্ত্রীর তিরস্কার, বিএনপির লজ...",হাজারো খবরের মাঝে এখন শরিয়তপুরের নড়িয়ায় চলছে ক...,1
2,633,channeldhaka.news,2019-05-04T19:59:48+00:00,Crime,সীমান্তে কাঁটাতারের বেড়া চুরি করে বিক্রি করছে ...,নাফ নদীর তীর ঘেসে অবৈধ অস্ত্র এবং চোরাকারবারিদ...,0
3,310,earki.com,"১৮:১৭, জানুয়ারি ৩১, ২০১৯",Miscellaneous,'টেন ইয়ারস চ্যালেঞ্জ' অনলাইন ট্রেন্ডকে একটু বে...,গত মঙ্গলবার ৩০ জানুয়ারি ফেসবুকে দেয়া এক পোস্টে...,0
4,597,somoynews.tv,2018-09-21 15:37:53,National,'সিনহার বই পরাজিত লোকের হা-হুতাশ',"ক্ষমতা হারানোর হতাশা থেকে, সাবেক প্রধান বিচারপ...",1


In [ ]:
print('The input text is:\n')
df['content'][0]

The input text is:



'নিজের প্রযোজিত সিনেমা ‘লাভরাত্রি’র নাম নিয়ে মামলা মুকদ্দমার ঝামেলায় পড়ে গিয়েছিলেন বলিউডের ভাইজান খ্যাত অভিনেতা ও প্রযোজক সালমান খান। শেষ পর্যন্ত বিতর্কের কাছে পরাজয় মেনে ঝামেলা এড়ালেন এই অভিনেতা। হিন্দু ধর্মানুভূতিতে আঘাত দেয়ার অভিযোগে সালমানের বিরুদ্ধে মামলা দায়ের হয়েছিলো গেল সপ্তাহেই। ‘লাভরাত্রি’ নামের সিনেমার প্রযোজক সালমান খান। আর এই ছবির নামকে হিন্দু ধর্মের অনুভূতিতে আঘাত হিসেবে দেখছেন বিশ্ব হিন্দু পরিষদ। আর সে কারণেই ভারতে ছবির মুক্তি ঠেকাতে এবং ছবির সঙ্গে জড়িতদের বিচারের দাবিতে মামলা দায়েরও করা হয়েছিলো সালমানকে। সরকারি পক্ষের উকিল বলছেন, দুর্গা পূজার মতো ধর্মীয় মহোৎসবকে ছোটো করার অভিপ্রায় নিয়েই ছবির নাম ‘লাভরাত্রি’ দেয়া হয়েছে। দূর্গা পূজার নবম রাত্রি অর্থ্যাৎ নবমীকে ভারতে ‘নবরাত্রি’ বলা হয়। এই বিষয়টিকে ধরে বছরের শুরুতে বিশ্ব হিন্দু পরিষদের নেতা অলোক কুমার বলেছিলেন পবিত্র ‘নবরাত্রি’কে বিকৃত করে এই নামের ব্যবহার উদ্দেশ্যপ্রণোদিত। আর এসব কারণে সালমানের বিরুদ্ধে মামলা হওয়ায় শেষ পর্যন্ত ‘লাভরাত্রি’ থেকে ছবির নাম পরিবর্তন করে রাখা হয়েছে ‘লাভযাত্রি’। ঝামেলা এড়াতে সালমান নিজেই টুইটারে 

In [ ]:
def preprocess(data):
    """This function is for preprocessing of the news contents. It removes punctuations, English characters and both of Bangla
    and English numerals. It tokenizes all the words and also removes stopwords.
    
    Args: 
        The Entire Dataframe
    Returns:
        Preprocessed news corpuses in a list of lists
    """
    corpus=[]
    for i in range(0, len(data)):
        x=re.sub('[^\u0980-\u09FF]',' ',data['content'][i])
        x=re.sub('[a-zA-Z0-9]+', ' ', x)
        x=re.sub('[০১২৩৪৫৬৭৮৯]+', ' ', x)
        x=x.split()
        x=[w for w in x if w not in set(stopwords)]
        x=' '.join(x)
        corpus.append(x)
    
    return corpus


corpus=preprocess(df)

In [ ]:
print('The cleaned text is:\n')
corpus[0]

The cleaned text is:



'প্রযোজিত সিনেমা লাভরাত্রি নাম মামলা মুকদ্দমার ঝামেলায় পড়ে গিয়েছিলেন বলিউডের ভাইজান খ্যাত অভিনেতা প্রযোজক সালমান খান শেষ বিতর্কের পরাজয় মেনে ঝামেলা এড়ালেন অভিনেতা হিন্দু ধর্মানুভূতিতে আঘাত দেয়ার অভিযোগে সালমানের বিরুদ্ধে মামলা দায়ের হয়েছিলো সপ্তাহেই লাভরাত্রি নামের সিনেমার প্রযোজক সালমান খান ছবির নামকে হিন্দু ধর্মের অনুভূতিতে আঘাত হিসেবে দেখছেন বিশ্ব হিন্দু পরিষদ কারণেই ভারতে ছবির মুক্তি ঠেকাতে ছবির জড়িতদের বিচারের দাবিতে মামলা দায়েরও হয়েছিলো সালমানকে সরকারি পক্ষের উকিল বলছেন দুর্গা পূজার ধর্মীয় মহোৎসবকে ছোটো অভিপ্রায় নিয়েই ছবির নাম লাভরাত্রি দেয়া হয়েছে দূর্গা পূজার নবম রাত্রি অর্থ্যাৎ নবমীকে ভারতে নবরাত্রি বিষয়টিকে বছরের শুরুতে বিশ্ব হিন্দু পরিষদের নেতা অলোক কুমার বলেছিলেন পবিত্র নবরাত্রি বিকৃত নামের উদ্দেশ্যপ্রণোদিত এসব কারণে সালমানের বিরুদ্ধে মামলা হওয়ায় শেষ লাভরাত্রি ছবির নাম পরিবর্তন হয়েছে লাভযাত্রি ঝামেলা এড়াতে সালমান টুইটারে ছবির পোস্টার শেয়ার বিষয়টি নিশ্চিত ক্যাপশনে লিখেছেন বানান ভুল লাভযাত্রি আসছে অক্টোবর ভারতে মুক্তি পেতে লাভযাত্রি সংশ্লিষ্টরা আশা প্রতিবাদ উঠবে ছবিটি বিতর্ক ছ

In [ ]:
hashing=HashingVectorizer(n_features=5000, ngram_range=(1, 3))
X=hashing.fit_transform(corpus).toarray()
y=df['label'].values
X.shape, y.shape

((8501, 5000), (8501,))

In [ ]:
print('The input vector after hashing is:\n')
print(X[0][0:30])

The input vector after hashing is:

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.         -0.01627361  0.01627361 -0.01627361
 -0.01627361  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]


In [ ]:
from tensorflow.keras.models import load_model
encoder=load_model('/content/drive/MyDrive/AutoEncoders/encoder_eff.h5')

In [ ]:
X_encode=encoder.predict(X)

In [ ]:
print('The input vector after autoencoder is:\n')
print(X_encode[0][0:30])

The input vector after autoencoder is:

[-0.00963157  0.00158157 -0.011741    0.00443562  0.00197384  0.00435493
 -0.00151328  0.00204745 -0.00148718 -0.00306128 -0.00117763 -0.00167098
 -0.00135604  0.00085981 -0.00505425 -0.01284218 -0.00900247  0.00240877
  0.00582511  0.00793516  0.00863674  0.0034942  -0.00464037  0.00045199
  0.00503031  0.0007014  -0.00260851 -0.00022047 -0.00416324 -0.00901561]


In [ ]:
lr_pred=lr_model.predict(X)

y=['Fake' if x==0 else 'Real' for x in y]
lr_pred=['Fake' if x==0 else 'Real' for x in lr_pred]

print(f'Actual Outcome ---> {y[0]} \n')
print(f'Prediction Outcome ---> {lr_pred[0]}')

Actual Outcome ---> Real 

Prediction Outcome ---> Real


In [ ]:
df[df['label'] == 0].head(20)

,articleID,domain,date,category,headline,content,label
2,633,channeldhaka.news,2019-05-04T19:59:48+00:00,Crime,সীমান্তে কাঁটাতারের বেড়া চুরি করে বিক্রি করছে ...,নাফ নদীর তীর ঘেসে অবৈধ অস্ত্র এবং চোরাকারবারিদ...,0
3,310,earki.com,"১৮:১৭, জানুয়ারি ৩১, ২০১৯",Miscellaneous,'টেন ইয়ারস চ্যালেঞ্জ' অনলাইন ট্রেন্ডকে একটু বে...,গত মঙ্গলবার ৩০ জানুয়ারি ফেসবুকে দেয়া এক পোস্টে...,0
6,1097,earki.com,2019-03-14T02:33:32+00:00,Lifestyle,"মেধাবীরা দেরিতে ঘুমায়, গালি দেয় বেশি : বলছে ...",আপনি কি অন্যদের তুলনায় একটু বেশিই গালি দেন? এ...,0
8,164,dailyinqilab.com,2018-03-20 12:00:00,Politics,শেখ হাসিনা বিশ্বের দ্বিতীয় সেরা প্রধানমন্ত্রী ...,সিঙ্গাপুরভিত্তিক গবেষণা প্রতিষ্ঠান ‘দ্য স্টাটি...,0
10,327,earki.com,"১৯:৪৫, ফেব্রুয়ারি ০৪, ২০১৯",Miscellaneous,"ঈশ্বরের দোহাই, আমার কোনো ভাস্কর্য বানাবেন না :...",মিয়ানমার থেকে বাংলাদেশে আশ্রয় নেয়া রোহিঙ্গা শর...,0
17,393,bengalbeats.com,2019-02-10T18:45:26+00:00,Miscellaneous,"শুধুমাত্র লস এঞ্জেলেস ও প্যারিস নয়, ঢাকাকে বিশ...","সময় পাল্টাচ্ছে, দেশ এগিয়ে যাচ্ছে তড়তড়িয়ে। বদল...",0
18,274,bengalbeats.com,2018-05-30T07:35:47+00:00,Miscellaneous,ছেলেদের মস্তিষ্ক কিভাবে কাজ করে জানেন কি? এই ১...,সাধারণত ছেলেদের জীবনে প্রত্যাশার পরিধি খুবই ছো...,0
19,1007,channeldhaka.news,2018-10-10T11:55:37+00:00,Entertainment,হারমোনিয়াম মার্কায় নির্বাচন করতে চান মাহফুজু...,আগামী জাতীয় নির্বাচনে হারমোনিয়াম মার্কায় প্...,0
24,698,earki.com,"১৯:০৩, জানুয়ারি ০২, ২০১৯",Miscellaneous,যেখানে সেখানে বিসিএসের বিদ্যা দেখাতে গেলে যেমন...,"ইদানিং দেশ-বিদেশ, সমাজ-রাজনীতি, সাহিত্য-সংস্কৃ...",0
26,765,mujibsenanews.com,2017-11-22 12:27:00,National,বিশ্বের তৃতীয় সৎ সরকার প্রধান শেখ হাসিনা,প্যারাডাইস পেপার্স আর পানামা পেপার্সের পর এবার...,0


In [ ]:
print('The input text is:\n')
df['content'][10]

The input text is:



'মিয়ানমার থেকে বাংলাদেশে আশ্রয় নেয়া রোহিঙ্গা শরণার্থীদের পরিদর্শন করতে কক্সবাজারে এসেছেন বিখ্যাত হলিউড অভিনেত্রী অ্যাঞ্জেলিনা জোলি। জাতিসংঘের শরণার্থী বিষয়ক সংস্থার (ইউএনএইচসিআর) বিশেষ দূত হিসেবে ৪ ফেব্রুয়ারি তিনি এই চার দিনের সফরে এসেছেন। জোলির মতো জনপ্রিয় তারকা বাংলাদেশে এসেছেন, তা নিয়ে হইচই পড়ে যাওয়াটাই স্বাভাবিক। কিন্তু হইচই এড়িয়ে তিনি নাকি থাকতে চান লোকচক্ষুর আড়ালে। eআরকিকে দেয়া বিশেষ এক সাক্ষাৎকারে এমনটাই জানালেন বিশ্বখ্যাত এই হলিউড তারকা।জোলি দেশে আসার খবর পাওয়া মাত্র আমরা তার মেসেঞ্জারে নক দেই। কিন্তু প্রথমে আমাদের (বুঝেন? যেন তেন কারো না, আমাদের!) মেসেজও সিন করেননি তিনি। অবশেষে ভিডিও কল দেয়া হলে তিনি ভিডিও অপশন অফ রেখে শুধু অডিও মাধ্যমে আলাপ চালান।এই ফোনালাপে ভীত সন্ত্রস্ত কন্ঠে তিনি বলেন, ‘বাংলাদেশে আসার আগে দেশে কী চলছে না চলছে, ফেসবুকে খোঁজ নিয়েছি। আসার প্ল্যান যখন চূড়ান্ত, তখনই দেখলাম ঢাকার গুলশানে সেলিব্রেটি গ্যালারি নামক এক বস্তু খুলেছে। সেখানে অবশ্য কোনো সেলিব্রেটিকেই আমি চিনতে পারিনি। ফেসবুকে আলোচনা থেকে বুঝলাম, এগুলো নাকি মেসি, শাহরুখ খান, মাইকেল জ্যাকসন এইসব বিখ্যাত 

In [ ]:
print('The cleaned text is:\n')
corpus[10]

The cleaned text is:



'মিয়ানমার বাংলাদেশে আশ্রয় নেয়া রোহিঙ্গা শরণার্থীদের পরিদর্শন কক্সবাজারে এসেছেন বিখ্যাত হলিউড অভিনেত্রী অ্যাঞ্জেলিনা জোলি জাতিসংঘের শরণার্থী বিষয়ক সংস্থার ইউএনএইচসিআর দূত হিসেবে ফেব্রুয়ারি দিনের সফরে এসেছেন জোলির জনপ্রিয় তারকা বাংলাদেশে এসেছেন হইচই পড়ে যাওয়াটাই স্বাভাবিক হইচই এড়িয়ে থাকতে লোকচক্ষুর আড়ালে আরকিকে দেয়া এক সাক্ষাৎকারে এমনটাই জানালেন বিশ্বখ্যাত হলিউড তারকা জোলি দেশে আসার খবর পাওয়া মেসেঞ্জারে নক দেই প্রথমে বুঝেন তেন কারো মেসেজও সিন করেননি অবশেষে ভিডিও কল দেয়া ভিডিও অপশন অফ অডিও আলাপ চালান ফোনালাপে ভীত সন্ত্রস্ত কন্ঠে বাংলাদেশে আসার দেশে চলছে চলছে ফেসবুকে খোঁজ নিয়েছি আসার প্ল্যান চূড়ান্ত তখনই দেখলাম ঢাকার গুলশানে সেলিব্রেটি গ্যালারি নামক এক বস্তু খুলেছে সেলিব্রেটিকেই চিনতে পারিনি ফেসবুকে আলোচনা বুঝলাম এগুলো মেসি শাহরুখ খান মাইকেল জ্যাকসন এইসব বিখ্যাত মানুষের ভাস্কর্য এরপরই সিদ্ধান্ত নিই বাংলাদেশে আসলেও মুখ দেখিয়ে লোকচক্ষুর আড়ালে থাকার করবো একজন সেলিব্রেটি হিসেবে ভাস্কর্যপূর্ণ পরিস্থিতিতে বাংলাদেশে আসাকেই ক্যারিয়ারের সবচেয়ে বড় ঝুঁকি আখ্যায়িত জানান অ্যাকশান সিনেমা সময় ঝুঁকি করেছি

In [ ]:
print('The input vector after hashing is:\n')
print(X[10][0:30])

The input vector after hashing is:

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -0.04522156  0.
  0.          0.          0.          0.          0.          0.
 -0.04522156  0.          0.          0.          0.          0.        ]


In [ ]:
print('The input vector after autoencoder is:\n')
print(X_encode[10][0:30])

The input vector after autoencoder is:

[ 0.00011098  0.00107366  0.00344558 -0.00017459  0.00166797 -0.00109933
 -0.00434442  0.00319853 -0.0007295  -0.00159404  0.0036609  -0.00299533
  0.00203265 -0.0024969   0.00152115 -0.00449073 -0.00252833  0.00685784
  0.01012982  0.00182164 -0.00264267  0.00609713  0.00014904  0.00302272
 -0.0058117  -0.00069131 -0.002502    0.00085154 -0.00371364 -0.00036071]


In [ ]:
print(f'Actual Outcome ---> {y[10]} \n')
print(f'Prediction Outcome ---> {lr_pred[10]}')

Actual Outcome ---> Fake 

Prediction Outcome ---> Fake
